In [ ]:
python main.py --model baffo32/decapoda-research-llama-7B-hf -- prune_method wanda --sparsity_ratio 0.5 --sparsity_type unstructured --save /content/drive/MyDrive/eshaan --save_model /content/drive/MyDrive/eshaan

In [ ]:
pip install transformers datasets wandb sentencepiece accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [ ]:
pip install llama-tokenizer

ERROR: Could not find a version that satisfies the requirement llama-tokenizer (from versions: none)
ERROR: No matching distribution found for llama-tokenizer


In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, LlamaTokenizer
from safetensors.torch import load_file
#from llama_tokenizer import LlamaTokenizer


# Load the validation dataset
validation_data = pd.read_parquet('/content/drive/MyDrive/validation-00000-of-00001.parquet')


print(f"Validation set size: {len(validation_data)}")



Validation set size: 1221


In [ ]:

# Register the custom tokenizer with the Transformers library
AutoTokenizer.register("llama_tokenizer", LlamaTokenizer)

# Path to the directory containing your model files
pruned_model_dir = '/content/drive/MyDrive/pruned_models/mistral_66_pruned'


tokenizer = LlamaTokenizer.from_pretrained(pruned_model_dir)
# Load the tokenizer
#tokenizer = AutoTokenizer.from_pretrained(pruned_model_dir)

In [ ]:

# Initialize an empty state dict
state_dict = {}

# Load each part of the model and update the state_dict
for i in range(1, 4):  # Assuming 3 parts: finalmodel1.safetensors, finalmodel2.safetensors, finalmodel3.safetensors
    part_state_dict = load_file(f"{pruned_model_dir}/model-0000{i}-of-00003.safetensors")
    state_dict.update(part_state_dict)



In [ ]:
# Load the model with the loaded state_dict
pruned_model = AutoModelForCausalLM.from_pretrained(pruned_model_dir, state_dict=state_dict).to('cuda')

# Ensure the pruned model handles padding correctly
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
pruned_model.config.pad_token_id = tokenizer.pad_token_id

print(f"Pruned model loaded successfully from {pruned_model_dir}.")

Pruned model loaded successfully from /content/drive/MyDrive/pruned_models/mistral_66_pruned.


In [ ]:
# Example input to check if the model is working
input_text = "The capital of France is"
inputs = tokenizer(input_text, return_tensors="pt").to('cuda')

# Generate output using the pruned model
with torch.no_grad():
    outputs = pruned_model.generate(inputs['input_ids'], max_length=20)

# Decode and print the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text:", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated text: The capital of France is Paris. Paris is known for its beautiful architecture, museums, and


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import torch

def evaluate_metrics_commonsenseqa(model, tokenizer, dataset):
    y_true = []
    y_pred = []

    for _, item in dataset.iterrows():
        question = item['question']
        choices = item['choices']['text']
        correct_answer = item['answerKey']

        # Skip if the correct answer is empty
        if not correct_answer:
            continue

        scores = []
        for choice in choices:
            # Prepare the input with the question and choice
            input_text = f"{question} {choice}"
            inputs = tokenizer(input_text, return_tensors="pt").to('cuda')

            # Generate the output logits
            with torch.no_grad():
                outputs = model(**inputs)

            # Calculate the score (likelihood) for the choice
            logits = outputs.logits[:, :-1, :]
            target_ids = inputs['input_ids'][:, 1:]
            loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
            loss = loss_fct(logits.permute(0, 2, 1), target_ids)
            score = -loss.sum().item()  # Negative log-likelihood as the score
            scores.append(score)

        # Select the choice with the highest score
        predicted_index = torch.argmax(torch.tensor(scores)).item()

        # Append true and predicted labels
        y_true.append(ord(correct_answer) - ord('A'))
        y_pred.append(predicted_index)

    # Calculate metrics
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)

    return accuracy, precision, recall, f1

# Evaluate on the validation set
accuracy, precision, recall, f1 = evaluate_metrics_commonsenseqa(pruned_model, tokenizer, validation_data)

print(f"Validation Accuracy: {accuracy * 100:.2f}%")
print(f"Validation Precision: {precision * 100:.2f}%")
print(f"Validation Recall: {recall * 100:.2f}%")
print(f"Validation F1-Score: {f1 * 100:.2f}%")

# import torch.nn.functional as F
# import math

# def calculate_perplexity(model, tokenizer, dataset):
#     total_loss = 0.0
#     total_tokens = 0

#     for _, item in dataset.iterrows():
#         question = item['question']
#         choices = item['choices']['text']

#         # Prepare the input with the question and the correct answer
#         input_text = f"{question} {choices[ord(item['answerKey']) - ord('A')]}"  # Using the correct answer as target
#         inputs = tokenizer(input_text, return_tensors="pt").to('cuda')

#         # Generate the output logits
#         with torch.no_grad():
#             outputs = model(**inputs)

#         logits = outputs.logits[:, :-1, :].contiguous().view(-1, outputs.logits.size(-1))
#         target_ids = inputs['input_ids'][:, 1:].contiguous().view(-1)

#         # Calculate negative log-likelihood
#         loss = F.cross_entropy(logits, target_ids, reduction='sum')
#         total_loss += loss.item()
#         total_tokens += target_ids.numel()

#     # Calculate and return perplexity
#     perplexity = math.exp(total_loss / total_tokens)
#     return perplexity

# # Calculate perplexity on the validation set
# validation_perplexity = calculate_perplexity(pruned_model, tokenizer, validation_data)
# print(f"Validation Perplexity: {validation_perplexity:.2f}")

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Validation Accuracy: 37.51%
Validation Precision: 37.50%
Validation Recall: 37.51%
Validation F1-Score: 37.45%
